Mounted at /content/drive


# Preparando o Ambiente

In [ ]:
!pip install validate-docbr

In [ ]:
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize
from validate_docbr import CPF, CNPJ
import re

In [ ]:
urlCompanies = "https://bigboost.bigdatacorp.com.br/companies"

1000


# Criando a classe para encontrar os Emails

In [ ]:
class BuscaCNPJ:

  def __init__(self,cnpj):
    cnpj = str(cnpj)
    self.cnpj = cnpj

  
  def CnpjFormatado(self):
    cnpj_me = cnpj.zfill(14)
    cnpj_me = cnpj_me.replace(".","")
    cnpj_me = cnpj_me.replace("-","")
    cnpj_me = cnpj_me.replace("/","")
    return cnpj_me

  def BuscaCNPJ(self):
    accessToken = 'accesstoken'
    payload = json.dumps({"Datasets": "related_people_emails","q": "doc"+"{"+f"{self.cnpj}"+"}","AccessToken": f"{accessToken}"})
    headers = {'Content-Type': 'application/json'}
    response = requests.request("POST", urlCompanies, headers=headers, data=payload)
    a = str(response.json()).strip()
    padrãoEmail = "([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)"
    respostasEmail= re.findall(padrãoEmail, a)
    respostasEmail = respostasEmail[0:2]
    respostasEmail = str(respostasEmail)
    return respostasEmail

  def __str__(self):
    return self.BuscaCNPJ()
  
  def __repr__(self):
    return self.BuscaCNPJ()

# Criando o metodo para encontrar os Nomes

In [ ]:
  def BuscaNome(cnpj):
    accessToken = 'accesstoken'
    payload = json.dumps({"Datasets": "relationships","q": "doc"+"{"+f"{cnpj}"+"}","AccessToken": f"{accessToken}"})
    headers = {'Content-Type': 'application/json'}
    response = requests.request("POST", urlCompanies, headers=headers, data=payload)
    b = str(response.json()).strip()
    osvaldin = b.find('RelatedEntityName') + len('RelatedEntityName') +4 #carcater inicial
    osvaldinpai = b.find('RelationshipType') -4 #caracter final
    nomeum = b[osvaldin:osvaldinpai]

    osvaldinSegundo = b.find('RelatedEntityName', osvaldin) + len("'RelatedEntityName': ")
    osvaldinpaiSegundo = b.find(", 'RelationshipType'",osvaldinpai+2)
    nomedois = b[osvaldinSegundo:osvaldinpaiSegundo]
    return nomeum,nomedois

# Percorrendo a lista de CNPJ

In [ ]:
arquivo = 'MapeamentoSUSEP - CSV (SUSEP pag 10 até 29).csv'

In [ ]:
lista = pd.read_csv(arquivo)

In [ ]:
lista = lista['CPF/CNPJ'].tolist()

In [ ]:
dic = {}
for c in lista:
  e = BuscaCNPJ(c)
  f = BuscaNome(c)
  dic[c] = e,f

In [ ]:
df = pd.DataFrame(dic.values(),columns=['Email','Nome'],index= dic.keys())

In [ ]:
dfModelagem = df.copy(deep=True)

In [ ]:
dfModelagem['Email'] = dfModelagem['Email'].apply(lambda x: str(x).replace("['",""))
dfModelagem['Email'] = dfModelagem['Email'].apply(lambda x: str(x).replace("']",""))
dfModelagem['Email'] = dfModelagem['Email'].apply(lambda x: str(x).replace("'",""))

dfModelagem['Nome'] = dfModelagem['Nome'].apply(lambda x: str(x).replace("(",""))
dfModelagem['Nome'] = dfModelagem['Nome'].apply(lambda x: str(x).replace(")",""))
dfModelagem['Nome'] = dfModelagem['Nome'].apply(lambda x: str(x).replace("'",""))

In [ ]:
dfModelagem.head()

,Email,Nome
18.798.075/0001-25,"drycafonseca@yahoo.com.br, adriana.campinas@se...","ADRIANA CRISTINA FONSECA, ""RAFAEL DE FREITAS P..."
41.050.436/0001-72,"anibalsouza@hotmail.com, anibalsouza@veloxmail...","ALEXANDRE DE SOUZA, ""ANIBAL DE SOUZA"""
02.972.899/0001-30,"registrobr@sunrise.com.br, ataluzia@hotmail.com","PAULO CESAR TEIXEIRA, ""LUZIA RODRIGUES DA SILV..."
21.367.663/0001-19,"josebva93@hotmail.com, zattusstardust@gmail.com","ANGELICA BORGES VEADO DE ASSIS CARVALHO, ""JOSE..."
51.813.343/0001-06,"bezerra_omni@superig.com.br, rosangelafernande...","ANTONIO CARLOS AZALIS BEZERRA, ""LEONARDO MORAL..."
